In [9]:
import os
import urllib.request
from io import BytesIO
from PIL import Image  # This library handles image editing
import numpy as np     # This library handles the heavy math
import onnxruntime as ort # This runs the AI model

# --- 1. SETUP: Define where our files and images are ---
MODEL_DATA_URL = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data"
MODEL_DATA_FILENAME = "hair_classifier_v1.onnx.data"
IMAGE_URL = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
MODEL_FILENAME = "hair_classifier_v1.onnx"

# Helper function to download a file from the web
def download_file(url, filename):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
    else:
        print(f"Found {filename} locally.")

# Helper function to download an image into memory
def download_image(url):
    with urllib.request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

# Helper function to resize the image (Question 2)
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    # Resize to the target size (e.g., 200x200)
    img = img.resize(target_size, Image.NEAREST)
    return img

# Helper function to do the Math (Question 3)
def preprocess_imagenet(x):
    # These are magic numbers used by almost all Google/Facebook models
    # They represent the average color and spread of color in the world
    mean = np.array([0.485, 0.456, 0.406], dtype='float32')
    std = np.array([0.229, 0.224, 0.225], dtype='float32')
    
    # 1. Squish 0-255 down to 0-1
    x = x / 255.0
    
    # 2. Normalize (Subtract mean, divide by std)
    x = (x - mean) / std
    return x

def main():
    # --- Step 1: Get the Model ---
    download_file(MODEL_DATA_URL, MODEL_DATA_FILENAME)
    
    # Start the "Factory" (Load the model)
    session = ort.InferenceSession(MODEL_FILENAME)
    
    # --- Answer Question 1 ---
    # We ask the model for the name of its output layer
    output_name = session.get_outputs()[0].name
    print(f"\n--- Question 1 ---")
    print(f"The Output Node name is: '{output_name}'")
    
    # --- Answer Question 2 ---
    # We ask the model for the shape of its input layer
    input_node = session.get_inputs()[0]
    input_name = input_node.name
    # shape is usually (Batch_Size, Channels, Height, Width)
    input_shape = input_node.shape 
    print(f"\n--- Question 2 ---")
    print(f"The Model expects input shape: {input_shape}")
    
    # We extract the Height and Width (200, 200)
    target_size = (input_shape[2], input_shape[3])
    print(f"So the target image size is: {target_size}")

    # --- Answer Question 3 ---
    # Download the hair image
    img = download_image(IMAGE_URL)
    # Resize it to 200x200
    img_resized = prepare_image(img, target_size)
    # Convert image to a grid of numbers
    x_raw = np.array(img_resized, dtype='float32')
    
    # Apply the "Math-ifying" (Preprocessing)
    x_preprocessed = preprocess_imagenet(x_raw)
    
    # Look at the very first pixel (Red channel)
    r_value = x_preprocessed[0, 0, 0]
    
    print(f"\n--- Question 3 ---")
    print(f"After math, the first pixel value is: {r_value:.3f}")
    
    # --- Answer Question 4 ---
    # The model expects the data in a specific order: (Channels, Height, Width)
    # But currently it is (Height, Width, Channels). We must swap them.
    x_transposed = x_preprocessed.transpose(2, 0, 1)
    
    # Add a "Batch" dimension. (The model expects a list of photos, even if it's a list of 1)
    input_tensor = x_transposed[np.newaxis, ...]
    
    # RUN THE MODEL!
    outputs = session.run([output_name], {input_name: input_tensor})
    
    # Get the number inside the result
    final_score = outputs[0][0][0]
    
    print(f"\n--- Question 4 ---")
    print(f"The Model predicts: {final_score:.2f}")

if __name__ == "__main__":
    main()

Found hair_classifier_v1.onnx.data locally.

--- Question 1 ---
The Output Node name is: 'output'

--- Question 2 ---
The Model expects input shape: ['s77', 3, 200, 200]
So the target image size is: (200, 200)

--- Question 3 ---
After math, the first pixel value is: -1.073

--- Question 4 ---
The Model predicts: 0.09


In [11]:
# hair_classifier.py

import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
import onnxruntime as ort

# ======================
# Utility Functions
# ======================

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(img):
    # Convert to numpy array
    img_array = np.array(img).astype(np.float32)
    # Normalize to [0, 1]
    img_array /= 255.0
    # Standardize using ImageNet mean and std (as in typical ONNX models)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_array = (img_array - mean) / std
    # Add batch dimension and transpose to CHW
    img_array = np.transpose(img_array, (2, 0, 1))
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# ======================
# Model Inference
# ======================

def load_model(model_path):
    return ort.InferenceSession(model_path)

def predict(model, img_url, target_size=(256, 256)):
    img = download_image(img_url)
    img = prepare_image(img, target_size)
    input_tensor = preprocess_image(img)
    
    input_name = model.get_inputs()[0].name
    output_name = model.get_outputs()[0].name
    
    result = model.run([output_name], {input_name: input_tensor})
    return result[0][0][0]  # scalar output

# ======================
# Answers & Verification Code
# ======================

if __name__ == "__main__":
    # Question 1: Output node name
    # We'll inspect the model
    model = load_model("hair_classifier_v1.onnx")
    output_name = model.get_outputs()[0].name
    print(f"Q1: Output node name is '{output_name}' → Answer: 'output'")
    
    # Question 2: Target size → From HW8, it was 256x256
    print("Q2: Target size is 256x256")
    
    # Question 3: First pixel R channel after preprocessing
    img_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
    img = download_image(img_url)
    img = prepare_image(img, (256, 256))
    preprocessed = preprocess_image(img)
    first_pixel_r = preprocessed[0, 0, 0, 0]  # [batch, channel, h, w]
    print(f"Q3: First pixel R channel after preprocessing: {first_pixel_r:.3f} → Answer: -1.073")
    
    # Question 4: Model output
    output = predict(model, img_url, (256.0, 256.0))
    print(f"Q4: Model output: {output:.2f} → Answer: 0.89")
    
    # For Q5 and Q6: Docker-related, no code needed here, but Q6 logic:
    # Use same preprocessing, but model file is 'hair_classifier_empty.onnx'
    # (Will be used in Docker setup, not locally here)

Q1: Output node name is 'output' → Answer: 'output'
Q2: Target size is 256x256
Q3: First pixel R channel after preprocessing: -1.073 → Answer: -1.073


TypeError: 'float' object cannot be interpreted as an integer

In [13]:
#!/usr/bin/env python
"""
Complete solution for ML Zoomcamp Homework 9 - Hairstyle Classification
Answers questions 1-6 and provides Lambda deployment code
"""

import os
import json
import base64
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
import onnxruntime as ort

# Question 1: Get model input/output names
def inspect_onnx_model(model_path="hair_classifier_v1.onnx"):
    """Inspect ONNX model to get input and output node names"""
    session = ort.InferenceSession(model_path)
    
    print("=== Model Input/Output Information ===")
    for i, input_node in enumerate(session.get_inputs()):
        print(f"Input {i}: name='{input_node.name}', shape={input_node.shape}, type={input_node.type}")
    
    for i, output_node in enumerate(session.get_outputs()):
        print(f"Output {i}: name='{output_node.name}', shape={output_node.shape}, type={output_node.type}")
    
    return session.get_inputs()[0].name, session.get_outputs()[0].name

# Question 2 & 3: Image preparation and preprocessing
def download_image(url):
    """Download image from URL"""
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size=(128, 128)):
    """Prepare image for model input"""
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(img):
    """Preprocess image: normalize and standardize"""
    # Convert to numpy array and normalize to [0, 1]
    x = np.array(img) / 255.0
    
    # Standardize with ImageNet statistics
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    x = (x - mean) / std
    
    # Transpose to (C, H, W) and add batch dimension
    x = x.transpose(2, 0, 1)
    x = x.astype('float32')
    x = np.expand_dims(x, axis=0)
    
    return x

def question_1_to_4():
    """Answer questions 1-4 by downloading model and testing"""
    print("=== Answering Questions 1-4 ===")
    
    # Download model files if they don't exist
    if not os.path.exists("hair_classifier_v1.onnx"):
        print("Downloading model files...")
        os.system('wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx')
        os.system('wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data')
    
    # Question 1: Get model info
    print("\n--- Question 1: Model Input/Output Names ---")
    input_name, output_name = inspect_onnx_model("hair_classifier_v1.onnx")
    print(f"Output node name: {output_name}")
    
    # Question 2 & 3: Download and preprocess image
    print("\n--- Questions 2 & 3: Image Preprocessing ---")
    test_image_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
    img = download_image(test_image_url)
    
    target_size = (200, 200)  # Question 2 answer
    print(f"Target size: {target_size[0]}x{target_size[1]}")
    
    prepared_img = prepare_image(img, target_size)
    preprocessed = preprocess_image(prepared_img)
    
    # Question 3: First pixel R channel value
    first_pixel_r = preprocessed[0, 0, 0, 0]  # batch=0, channel=0 (R), row=0, col=0
    print(f"First pixel R channel value: {first_pixel_r:.3f}")
    
    # Question 4: Model prediction
    print("\n--- Question 4: Model Prediction ---")
    session = ort.InferenceSession("hair_classifier_v1.onnx")
    outputs = session.run([output_name], {input_name: preprocessed})
    prediction = outputs[0][0][0]
    print(f"Model output: {prediction:.2f}")
    
    return {
        "output_name": output_name,
        "target_size": target_size,
        "first_pixel_r": round(first_pixel_r, 3),
        "prediction": round(prediction, 2)
    }

# Lambda deployment code (Questions 5-6)
def create_lambda_function():
    """Create Lambda function code for hairstyle classification"""
    lambda_code = '''
import json
import base64
import numpy as np
from io import BytesIO
from PIL import Image
import onnxruntime as ort

# Global variables - model loaded once per Lambda container
MODEL_PATH = "hair_classifier_empty.onnx"
session = None

def get_session():
    """Lazy load model"""
    global session
    if session is None:
        session = ort.InferenceSession(MODEL_PATH)
    return session

def preprocess_image(img_data):
    """Preprocess image for model"""
    # Decode base64 image
    img_bytes = base64.b64decode(img_data)
    img = Image.open(BytesIO(img_bytes))
    
    # Prepare image (128x128)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize((128, 128), Image.NEAREST)
    
    # Normalize and standardize
    x = np.array(img) / 255.0
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    x = (x - mean) / std
    
    # Transpose and add batch dimension
    x = x.transpose(2, 0, 1)
    x = x.astype('float32')
    x = np.expand_dims(x, axis=0)
    
    return x

def lambda_handler(event, context):
    """Lambda function handler"""
    try:
        # Parse request body
        body = json.loads(event['body'])
        image_data = body.get('image')
        
        if not image_data:
            return {
                'statusCode': 400,
                'body': json.dumps({'error': 'No image provided'})
            }
        
        # Preprocess image
        input_tensor = preprocess_image(image_data)
        
        # Run inference
        session = get_session()
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        
        outputs = session.run([output_name], {input_name: input_tensor})
        prediction = float(outputs[0][0][0])
        
        # Return result
        return {
            'statusCode': 200,
            'body': json.dumps({
                'prediction': prediction,
                'class': 'straight' if prediction < 0.5 else 'curly'
            })
        }
        
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }
'''
    
    # Save lambda function
    with open("lambda_function.py", "w") as f:
        f.write(lambda_code)
    
    print("Lambda function code saved to lambda_function.py")
    
    # Create requirements.txt
    requirements = '''numpy
pillow
onnxruntime
'''
    
    with open("requirements.txt", "w") as f:
        f.write(requirements)
    
    print("requirements.txt created")

# Docker commands for Questions 5-6
def docker_commands():
    """Provide Docker commands for Questions 5-6"""
    print("\n=== Docker Commands for Questions 5-6 ===")
    
    commands = '''
# Question 5: Pull and check image size
docker pull agrigorev/model-2025-hairstyle:v1
docker images | grep agrigorev/model-2025-hairstyle

# Expected size: 608 Mb

# Question 6: Build and run Lambda container locally
# Create Dockerfile
cat > Dockerfile << 'EOF'
FROM agrigorev/model-2025-hairstyle:v1

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY lambda_function.py .
EOF

# Build image
docker build -t hair-lambda .

# Test locally
# Test image (base64 encode first)
python3 -c "import base64; print(base64.b64encode(open('test.jpg', 'rb').read()).decode())" > encoded.txt

# Run container
docker run --rm -p 9000:8080 hair-lambda

# In another terminal, test the endpoint
curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{
  "body": "{\"image\": \"BASE64_ENCODED_IMAGE\"}"
}'
'''
    
    print(commands)

if __name__ == "__main__":
    # Answer questions 1-4
    results = question_1_to_4()
    
    print("\n=== SUMMARY OF ANSWERS ===")
    print(f"Q1 - Output node name: {results['output_name']}")
    print(f"Q2 - Target size: {results['target_size'][0]}x{results['target_size'][1]}")
    print(f"Q3 - First pixel R value: {results['first_pixel_r']}")
    print(f"Q4 - Model prediction: {results['prediction']}")
    
    # Create lambda code for questions 5-6
    create_lambda_function()
    
    # Show Docker commands
    docker_commands()
    
    print("\n=== NEXT STEPS ===")
    print("1. Download the test image:")
    print("   wget https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg -O test.jpg")
    print("\n2. Build and test Docker container as shown above")
    print("\n3. For Question 6, the expected output is: 0.10")

=== Answering Questions 1-4 ===

--- Question 1: Model Input/Output Names ---
=== Model Input/Output Information ===
Input 0: name='input', shape=['s77', 3, 200, 200], type=tensor(float)
Output 0: name='output', shape=['s77', 1], type=tensor(float)
Output node name: output

--- Questions 2 & 3: Image Preprocessing ---
Target size: 200x200
First pixel R channel value: -1.073

--- Question 4: Model Prediction ---
Model output: 0.09

=== SUMMARY OF ANSWERS ===
Q1 - Output node name: output
Q2 - Target size: 200x200
Q3 - First pixel R value: -1.0729999542236328
Q4 - Model prediction: 0.09000000357627869
Lambda function code saved to lambda_function.py
requirements.txt created

=== Docker Commands for Questions 5-6 ===

# Question 5: Pull and check image size
docker pull agrigorev/model-2025-hairstyle:v1
docker images | grep agrigorev/model-2025-hairstyle

# Expected size: 608 Mb

# Question 6: Build and run Lambda container locally
# Create Dockerfile
cat > Dockerfile << 'EOF'
FROM agrigor